# Notebook on cracking geetest CAPTCHA 
# This Notebook is for educational only
# Author: Joe Ho
# Date: 5, January, 2020

## Import Library

In [ ]:
import cv2
import time
import numpy as np
import urllib.request

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from random import randint

from scipy import ndimage, misc
from scipy.spatial.distance import hamming

import pandas as pd

from selenium import webdriver

import importlib

import icon_matching
import icon_detection
import helpers

In [ ]:
# # Uncomment this part for real Geetest CAPTCHA


# image_path = ".//image//"+str(time.time())+".jpg"

# driver = webdriver.Chrome() 

# # driver.get("https://www.geetest.com/en")  # Old Path
# driver.get("https://www.geetest.com/show")  

In [ ]:
# # Uncomment this part for real Geetest CAPTCHA


# time.sleep(10)
# img = driver.find_element_by_class_name("geetest_item_img")
# src = img.get_attribute('src')

# # download the image
# urllib.request.urlretrieve(src, image_path)

In [ ]:
# # Comment this part for real Geetest CAPTCHA
image_name = 2
image_path = f"dataset/img/{image_name}.jpg" 

## Load geetest Image

In [ ]:
img_grey = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
plt.imshow(img_grey, cmap="gray")

## Load targets

In [ ]:
csv = pd.read_csv('dataset/data.csv', delimiter=',', header=None, index_col=0)
print(csv)
icons_name = csv.loc[image_name,1:]
icons_path = [f'dataset/icon/{icon_name}.png' for icon_name in icons_name]
icons_path

In [ ]:
def filter_correct_channel(img):
    if (img[:, :, 0] > 0).any(): return 255 - img[:, :, 0] #B
    if (img[:, :, 1] > 0).any(): return 255 - img[:, :, 1] #G
    if (img[:, :, 2] > 0).any(): return 255 - img[:, :, 2] #R
    return img[:, :, 3]

In [ ]:
def read_and_preprocess_icon(icon_path):
    icon = cv2.imread(icon_path, cv2.IMREAD_UNCHANGED)
    return filter_correct_channel(icon)

In [ ]:
targets = [read_and_preprocess_icon(icon_path) for icon_path in icons_path]

common_shape = np.min([target.shape for target in targets], axis=0)
targets = [target.reshape(common_shape) for target in targets]

for i, target in enumerate(targets):
    plt.subplot(1, len(targets), i + 1)
    plt.imshow(target, cmap="gray")
    print(f"shape of {icons_path[i]}: {target.shape}")

## Icons filter

In [ ]:
# Image preprocessing

color_threshold = 250
main_pane = img_grey # cv2.blur(img_grey,(3,3)) # By blurring, we can remove some white pixels which may affecting the matching
main_pane[main_pane<color_threshold] = 0
main_pane[main_pane>=color_threshold] = 255
plt.imshow(main_pane, cmap="gray")

## Locate icons

In [ ]:
importlib.reload(icon_detection)
icons_rect_coordinates = icon_detection.find_bounding_box(image=main_pane, bounding_box_lower_thresholds=(20,20), 
                                                          bounding_box_upper_thresholds=(100,100))
icons = icon_detection.segment_pictures(image=main_pane, rect_coordinates=icons_rect_coordinates, 
                                        dimension_resized=common_shape)

In [ ]:
for i, icon in enumerate(icons):
    plt.subplot(1, len(icons), i + 1)
    plt.imshow(icon, cmap="gray")
    print(icon.shape)

In [ ]:
importlib.reload(helpers)
helpers.draw_bounding_box(main_pane, icons_rect_coordinates)    

## Identify the most similiar icon in main pane for each target

### Compute similarity matrix

icon\target | t1 | t2 | t3 |
------------|----|----|----|
i1          |s11 |s12 |s13 |
i2          |s21 |s22 |s23 |
i3          |s31 |s32 |s33 |

1. Flatten the matrix. 
2. Use argsort on similarities in descending order and then pick the first nb_icons element
3. compute mapping based on mapping\[index \% nb_icons\] = index / nb_icons 

In [ ]:
importlib.reload(icon_matching)
mapping = icon_matching.match_icons_with_targets(icons=icons, targets=targets)
print(mapping)

### Plot result

In [ ]:
fig,ax = plt.subplots(1)
ax.imshow(main_pane, cmap="gray")

for key in range(len(targets)):
    x,y,w,h = icons_rect_coordinates[mapping[key]]
    
    # x,y is the coordinate of top left hand corner
    # Bounding box is 70x70, so centre of circle = (x+70/2, y+70/2), i.e. (x+35, y+35)
    centre_x = x+(w//2)
    centre_y = y+(h//2)
    # Plot circle
    circle = plt.Circle((centre_x,centre_y), 20, color="green", fill=False, linewidth=5)
    # Plot centre
    fontsize = 24
    plt.text(centre_x - fontsize//4, centre_y + fontsize//4, str(key), color="red", fontsize=fontsize)
    ax.add_patch(circle)
     
plt.show()

In [ ]:
for i, target in enumerate(targets):
    plt.subplot(1, len(targets), i + 1)
    plt.imshow(target, cmap="gray")

## Click targets

In [ ]:
# # Uncomment this part for real Geetest CAPTCHA


# time.sleep(10)
# for i, target in enumerate(targets):
#     key = i
#     x,y,w,h = icons_rect_coordinates[mapping[key]]
    
#     # x,y is the coordinate of top left hand corner
#     # Bounding box is 70x70, so centre of circle = (x+70/2, y+70/2), i.e. (x+35, y+35)
#     centre_x = x+(w//2)
#     centre_y = y+(h//2)
    
#     ele=driver.find_element_by_xpath(u"(.//*[normalize-space(text()) and normalize-space(.)='加载中...'])[1]/following::div[1]")
#     action = webdriver.common.action_chains.ActionChains(driver)
#     action.move_to_element_with_offset(ele, centre_x, centre_y)
#     time.sleep(randint(100,700)/1000) # Random Pause between two consecutive clicks
#     action.click()
#     action.perform()

# # Click Ok button in CAPTCHA after some random pause
# time.sleep(randint(100,700)/1000)
# driver.find_element_by_xpath(u"(.//*[normalize-space(text()) and normalize-space(.)='帮助反馈'])[1]/following::div[2]").click()    